In [1]:
%matplotlib inline
import utils;reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [2]:
path="data/redux/sample/"
model_path=path +'models/'
batch_size=64

In [3]:
model = vgg_ft(2)

In [4]:
import bcolz
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
def load_array(fname): return bcolz.open(fname)[:]

In [5]:
trn_data = load_array(model_path+'train_data.bc')
val_data = load_array(model_path+'valid_data.bc')

In [125]:
trn_features = load_array(model_path+'train_convlayer_features.bc')
val_features = load_array(model_path+'valid_convlayer_features.bc')

In [126]:
trn_features.shape

(266L, 512L, 14L, 14L)

In [7]:
layers=model.layers
opt = RMSprop(lr=0.1)
for layer in layers: layer.trainable=True
K.set_value(opt.lr, 0.001)

In [8]:
def fit_model(model, batches, val_batches, nb_epoch=1):
    model.fit_generator(batches, samples_per_epoch=batches.n, nb_epoch=nb_epoch, 
                        validation_data=val_batches, nb_val_samples=val_batches.n)

In [9]:
val_batches = get_batches(path+'valid', shuffle=False, batch_size=1)
batches = get_batches(path+'train', shuffle=False, batch_size=1)

Found 51 images belonging to 2 classes.
Found 266 images belonging to 2 classes.


In [16]:
fit_model(model, batches, val_batches, 2)

Epoch 1/2
266/266 [==============================] - 265s - loss: 0.2945 - acc: 0.9624 - val_loss: 0.2762 - val_acc: 0.9608
Epoch 2/2
266/266 [==============================] - 270s - loss: 0.1133 - acc: 0.9737 - val_loss: 0.1420 - val_acc: 0.9804


In [17]:
K.set_value(opt.lr, 0.01)

In [20]:
fit_model(model, batches, val_batches, 2)

Epoch 1/2
266/266 [==============================] - 267s - loss: 0.0137 - acc: 0.9962 - val_loss: 2.5790e-04 - val_acc: 1.0000
Epoch 2/2
266/266 [==============================] - 260s - loss: 0.0037 - acc: 1.0000 - val_loss: 0.0176 - val_acc: 0.9804


In [21]:
model.save_weights(model_path + 'finetune3.h5')

In [10]:
model.load_weights(model_path+'finetune3.h5')

In [22]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [11]:
last_conv_idx=[index for index,layer in enumerate(model.layers) if type(layer) is Convolution2D][-1]

In [12]:
last_conv_idx

30

In [13]:
conv_layers=model.layers[:last_conv_idx+1]

In [94]:
conv_model=Sequential(conv_layers)

In [56]:
conv_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
                                                                   lambda_1[0][0]                   
                                                                   lambda_1[0][0]                   
                                                                   lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]   

In [15]:
fc_layers=model.layers[last_conv_idx+1:]

In [16]:
batches=get_batches(path+'train',shuffle=False,batch_size=batch_size)
val_batches=get_batches(path+'valid',shuffle=False,batch_size=batch_size)

Found 266 images belonging to 2 classes.
Found 51 images belonging to 2 classes.


In [20]:
val_classes=val_batches.classes
trn_classes=batches.classes
val_labels=onehot(val_classes)
trn_labels=onehot(trn_classes)

In [18]:
val_features=conv_model.predict_generator(val_batches,val_batches.nb_sample)

In [19]:
trn_features=conv_model.predict_generator(batches,batches.nb_sample)

KeyboardInterrupt: 

In [ ]:
val_features

In [27]:
save_array(model_path + 'train_convlayer_features.bc', trn_features)
save_array(model_path + 'valid_convlayer_features.bc', val_features)

In [21]:
def proc_wgts(layer): return [o/2 for o in layer.get_weights()]

In [22]:
opt=RMSprop(lr=0.00001, rho=0.9)

In [23]:
conv_layers[-1].output_shape[1:]

(512, 14, 14)

In [24]:
def get_fc_model():
    model=Sequential([
            MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
            Flatten(),
            Dense(4096,activation='relu'),
            Dropout(0.),
            Dense(4096, activation='relu'),
            Dropout(0.),
            Dense(2, activation='softmax')
        ])
    for l1,l2 in zip(model.layers,fc_layers):
        l1.set_weights(proc_wgts(l2))
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model
    

In [25]:
fc_model=get_fc_model()

In [40]:
fc_model.fit(trn_features, trn_labels, nb_epoch=8, 
             batch_size=batch_size, validation_data=(val_features, val_labels))

Train on 266 samples, validate on 51 samples
Epoch 1/8
266/266 [==============================] - 10s - loss: 6.8859e-07 - acc: 1.0000 - val_loss: 0.0261 - val_acc: 0.9804
Epoch 2/8
266/266 [==============================] - 10s - loss: 5.8260e-07 - acc: 1.0000 - val_loss: 0.0260 - val_acc: 0.9804
Epoch 3/8
266/266 [==============================] - 10s - loss: 5.0574e-07 - acc: 1.0000 - val_loss: 0.0270 - val_acc: 0.9804
Epoch 4/8
266/266 [==============================] - 10s - loss: 4.1992e-07 - acc: 1.0000 - val_loss: 0.0275 - val_acc: 0.9804
Epoch 5/8
266/266 [==============================] - 10s - loss: 3.6390e-07 - acc: 1.0000 - val_loss: 0.0281 - val_acc: 0.9804
Epoch 6/8
266/266 [==============================] - 10s - loss: 3.1057e-07 - acc: 1.0000 - val_loss: 0.0280 - val_acc: 0.9804
Epoch 7/8
266/266 [==============================] - 10s - loss: 2.7696e-07 - acc: 1.0000 - val_loss: 0.0269 - val_acc: 0.9804
Epoch 8/8
266/266 [==============================] - 10s - loss: 2

In [41]:
fc_model.save_weights(model_path+'no_dropout.h5')

In [26]:
fc_model.load_weights(model_path+'no_dropout.h5')

In [27]:
gen = image.ImageDataGenerator(rotation_range=15, width_shift_range=0.1, 
                               height_shift_range=0.1, zoom_range=0.1, horizontal_flip=True)

In [28]:
batches = get_batches(path+'train', gen, batch_size=batch_size)
# NB: We don't want to augment or shuffle the validation set
val_batches = get_batches(path+'valid', shuffle=False, batch_size=batch_size)

Found 266 images belonging to 2 classes.
Found 51 images belonging to 2 classes.


In [57]:
for layer in conv_model.layers:
    layer.trainable=False
conv_model.add(fc_model)

In [58]:
conv_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
                                                                   lambda_1[0][0]                   
                                                                   lambda_1[0][0]                   
                                                                   lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]   

In [30]:
conv_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [31]:
conv_model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=8, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Epoch 1/8
266/266 [==============================] - 282s - loss: 0.0162 - acc: 0.9962 - val_loss: 8.3024e-05 - val_acc: 1.0000
Epoch 2/8
266/266 [==============================] - 247s - loss: 1.1166e-04 - acc: 1.0000 - val_loss: 0.0083 - val_acc: 1.0000
Epoch 3/8


KeyboardInterrupt: 

In [32]:
conv_model.save_weights(model_path + 'aug1.h5')

In [96]:
conv_model.load_weights(model_path + 'aug1.h5')

ValueError: You are trying to load a weight file containing 14 layers into a model with 13 layers.

In [102]:
conv_layers[-1].output_shape[1:]

(512, 14, 14)

In [103]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(p),
        BatchNormalization(),
        Dense(4096, activation='relu'),
        Dropout(p),
        BatchNormalization(),
        Dense(1000, activation='softmax')
    ]

In [39]:
p=0.6

In [88]:
bn_model=Sequential(get_bn_layers(p))

In [53]:
bn_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
maxpooling2d_8 (MaxPooling2D)    (None, 512, 7, 7)     0           maxpooling2d_input_2[0][0]       
____________________________________________________________________________________________________
flatten_4 (Flatten)              (None, 25088)         0           maxpooling2d_8[0][0]             
____________________________________________________________________________________________________
dense_11 (Dense)                 (None, 4096)          102764544   flatten_4[0][0]                  
____________________________________________________________________________________________________
dropout_7 (Dropout)              (None, 4096)          0           dense_11[0][0]                   
___________________________________________________________________________________________

In [41]:
bn_model.load_weights('vgg16_bn.h5')

ValueError: You are trying to load a weight file containing 18 layers into a model with 5 layers.

In [54]:
for layer in conv_model.layers:
    layer.trainable=False
conv_model.add(bn_model)

IndexError: tuple index out of range

In [51]:
conv_model.load_weights('vgg16_bn.h5')

ValueError: You are trying to load a weight file containing 18 layers into a model with 14 layers.

In [52]:
conv_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
                                                                   lambda_1[0][0]                   
                                                                   lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
                                                                   zeropadding2d_1[1][0]   

In [73]:
conv_model_bn=Sequential(conv_layers)

In [74]:
conv_model_bn.add(MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]))

In [75]:
conv_model_bn.add(Flatten())


In [76]:
conv_model_bn.add(Dense(4096, activation='relu'))
conv_model_bn.add(Dropout(0.6))
conv_model_bn.add(BatchNormalization())
conv_model_bn.add(Dense(4096, activation='relu'))
conv_model_bn.add(Dropout(0.6))
conv_model_bn.add(BatchNormalization())
conv_model_bn.add(Dense(1000, activation='softmax'))

In [77]:
conv_model_bn.summary(
)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
                                                                   lambda_1[0][0]                   
                                                                   lambda_1[0][0]                   
                                                                   lambda_1[0][0]                   
                                                                   lambda_1[0][0]                   
                                                                   lambda_1[0][0]          

In [78]:
conv_model_bn.load_weights('vgg16_bn.h5')

In [79]:
def proc_wgts(layer, prev_p, new_p):
    scal = (1-prev_p)/(1-new_p)
    return [o*scal for o in layer.get_weights()]

In [80]:
for l in conv_model_bn.layers: 
    if type(l)==Dense: l.set_weights(proc_wgts(l, 0.3, 0.6))

In [81]:
conv_model_bn.pop()

In [82]:
for layers in conv_model_bn.layers: layers.trainable=False

In [83]:
conv_model_bn.add(Dense(2,activation='softmax'))

In [84]:
conv_model_bn.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])

In [87]:
conv_model_bn.fit_generator(batches,samples_per_epoch=batches.n, nb_epoch=8, 
                        validation_data=val_batches, nb_val_samples=val_batches.n)

Epoch 1/8
266/266 [==============================] - 238s - loss: 1.1726 - acc: 0.6053 - val_loss: 0.4039 - val_acc: 0.8627
Epoch 2/8
266/266 [==============================] - 229s - loss: 0.5587 - acc: 0.8045 - val_loss: 0.0219 - val_acc: 1.0000
Epoch 3/8
266/266 [==============================] - 235s - loss: 0.2880 - acc: 0.8985 - val_loss: 0.0675 - val_acc: 0.9804
Epoch 4/8
266/266 [==============================] - 235s - loss: 0.1922 - acc: 0.9549 - val_loss: 0.1029 - val_acc: 0.9804
Epoch 5/8
266/266 [==============================] - 233s - loss: 0.2723 - acc: 0.9135 - val_loss: 0.0459 - val_acc: 0.9804
Epoch 6/8
266/266 [==============================] - 219s - loss: 0.1778 - acc: 0.9361 - val_loss: 0.0126 - val_acc: 1.0000
Epoch 7/8
266/266 [==============================] - 215s - loss: 0.1053 - acc: 0.9662 - val_loss: 0.0358 - val_acc: 0.9804
Epoch 8/8
266/266 [==============================] - 216s - loss: 0.1135 - acc: 0.9549 - val_loss: 0.0236 - val_acc: 0.9804


In [95]:
conv_model_bn.save_weights(model_path+'conv_bn.h5')

In [116]:
bn_model = Sequential(get_bn_layers(0.6))

In [117]:
bn_model.pop()
for l in bn_model.layers: 
    if type(l)==Dense: l.set_weights(proc_wgts(l, 0.3, 0.6))

In [118]:
bn_model.add(Dense(2,activation='softmax'))

In [119]:
bn_model.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])

In [120]:
bn_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
maxpooling2d_16 (MaxPooling2D)   (None, 512, 7, 7)     0           maxpooling2d_input_8[0][0]       
____________________________________________________________________________________________________
flatten_13 (Flatten)             (None, 25088)         0           maxpooling2d_16[0][0]            
____________________________________________________________________________________________________
dense_43 (Dense)                 (None, 4096)          102764544   flatten_13[0][0]                 
____________________________________________________________________________________________________
dropout_25 (Dropout)             (None, 4096)          0           dense_43[0][0]                   
___________________________________________________________________________________________

In [127]:
bn_model.fit(trn_features, trn_labels, nb_epoch=8, validation_data=(val_features, val_labels))

Train on 266 samples, validate on 51 samples
Epoch 1/8
266/266 [==============================] - 17s - loss: 0.3004 - acc: 0.8722 - val_loss: 1.5503 - val_acc: 0.8627
Epoch 2/8
266/266 [==============================] - 18s - loss: 0.0544 - acc: 0.9737 - val_loss: 0.3160 - val_acc: 0.9804
Epoch 3/8
266/266 [==============================] - 19s - loss: 0.0307 - acc: 0.9887 - val_loss: 0.4474 - val_acc: 0.9412
Epoch 4/8
266/266 [==============================] - 19s - loss: 0.0528 - acc: 0.9812 - val_loss: 0.6080 - val_acc: 0.9216
Epoch 5/8
266/266 [==============================] - 18s - loss: 0.0055 - acc: 1.0000 - val_loss: 0.4334 - val_acc: 0.9412
Epoch 6/8
266/266 [==============================] - 18s - loss: 0.1226 - acc: 0.9737 - val_loss: 0.8596 - val_acc: 0.9216
Epoch 7/8
266/266 [==============================] - 18s - loss: 0.0198 - acc: 0.9925 - val_loss: 1.0894 - val_acc: 0.9020
Epoch 8/8
266/266 [==============================] - 18s - loss: 0.0207 - acc: 0.9925 - val_lo

In [130]:
test_path = path + '/test/' 

batches, preds = bn_model.predict(test_path, batch_size = batch_size*2)

TypeError: Error when checking : data should be a Numpy array, or list/dict of Numpy arrays. Found: data/redux/sample//test/...

In [109]:
bn_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
maxpooling2d_14 (MaxPooling2D)   (None, 512, 7, 7)     0           maxpooling2d_input_6[0][0]       
____________________________________________________________________________________________________
flatten_11 (Flatten)             (None, 25088)         0           maxpooling2d_14[0][0]            
____________________________________________________________________________________________________
dense_35 (Dense)                 (None, 4096)          102764544   flatten_11[0][0]                 
____________________________________________________________________________________________________
dropout_21 (Dropout)             (None, 4096)          0           dense_35[0][0]                   
___________________________________________________________________________________________

In [122]:
trn_features.shape

(266L, 1000L)